In [8]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister, execute, Aer, IBMQ,BasicAer
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import math
import matplotlib.pyplot as plt
from random import *

In [9]:
#calcul le nombre de 1 dans la représentation binaire du nombre de qubit
def countOnes(numberOfQubits):
    counts = 0
    while numberOfQubits!=0:
        numberOfQubits=numberOfQubits&(numberOfQubits-1)
        counts=counts+1
    return counts

In [132]:
def buildAdder(numberOfQubits,valueA,valueB):
    
    
    logn=math.floor(math.log(numberOfQubits,2))
    log2_3n=math.floor(math.log((2/3)*numberOfQubits,2))
    nP_round1=math.floor(numberOfQubits/2)-1
    stateVectorSize=int(math.pow(2,numberOfQubits))
    #Definition of the registers
    
    n_auxillaryQubits=numberOfQubits-countOnes(numberOfQubits)-logn-nP_round1
    
    
    A = QuantumRegister(numberOfQubits,"a")
    circ_A=QuantumCircuit(A)
    B = QuantumRegister(numberOfQubits,"b")
    circ_B=QuantumCircuit(B)
    S=QuantumRegister(numberOfQubits+1,"s")
    if(n_auxillaryQubits>0):
        Anc=QuantumRegister(n_auxillaryQubits,"anc")
        circ_S=QuantumCircuit(S,Anc)
    else:
        circ_S=QuantumCircuit(S)
    
    AdderBis = circ_A+circ_B+circ_S
    
    #initializing circuit
    state_A = [0]*stateVectorSize #initialize to 0
    state_B = [0]*stateVectorSize #initialize to 0
    state_A[valueA-1]=1
    state_B[valueB-1]=1
    circ_A.initialize(state_A,A)
    circ_B.initialize(state_B,B)
    
    Adder = circ_A+circ_B+circ_S
    
    
    
    #Utility list
    Buffer=[]  #on y place les P successifs
    Buffer_Anc_list=[]
    P_list=[]
    precompute_list=[]     #On peut s'en passer aussi
    P_inv_list=[]
    Anc_list=[]
    
    #Precomputation for uncomputation before Prounds
    #On calcule les indices  auquelles on va
    #stocker la valeur des P-round1 pour pouvoir 
    #prévoir ou placer les cx de P0
    roundUpperBound = math.floor(numberOfQubits/2)         #Inutile on peut s'en servir juste à P0 pour 
    #prévoir ou mettre les cx
    for m in range(1,roundUpperBound):
        precompute_list.append(2*m+1)
                

    #compute g[i,i+1] into auxillary register for 0<i<n
    #G0 round
    for i in range(0,numberOfQubits):
        Adder.ccx(A[i],B[i],S[i+1])
        AdderBis.ccx(A[i],B[i],S[i+1])
        
    
        
    #Adder.barrier()    
    #compute p[i,i+1] into b register for 0<i<n
    #P0 round
    for i in range(1,numberOfQubits):
        if(i+1 not in precompute_list):
            #On ne place pas de cnot aux endroits ou on va devoir uncompute
            Adder.cx(A[i],B[i])
            AdderBis.cx(A[i],B[i])
             
    
    #Adder.barrier()
    i=0
    for t in range(1,logn+1):
        tmp=int(math.pow(2,t))
        tmp1=int(math.pow(2,t-1))
        roundUpperBound = math.floor(numberOfQubits/(tmp))
        #G-round
        #On effectue alterne les rounds entre G-round et P-round
        for m in range(0,roundUpperBound):
            if(t==1):
                #1er round on utilise la formule classique
                Adder.ccx(S[tmp*m+tmp1],B[2*m+1],S[tmp*m+tmp])
                AdderBis.ccx(S[tmp*m+tmp1],B[2*m+1],S[tmp*m+tmp])
            else:
                #Pour les rounds suivant on utilise 
                Adder.ccx(S[tmp*m+tmp1],P_list[t-2][2*m],S[tmp*m+tmp])
                AdderBis.ccx(S[tmp*m+tmp1],P_list[t-2][2*m],S[tmp*m+tmp])
        #Adder.barrier()
        #P-round
        for m in range(1,roundUpperBound):
            if(t==1):
                #Uncompute previous operation
                #On doit d'abord uncompute les opérations de g0
                
                #Uncomputation
                Adder.ccx(A[2*m],B[2*m],S[2*m+1])
                AdderBis.ccx(A[2*m],B[2*m],S[2*m+1])                 
                Adder.cx(A[2*m],B[2*m])  
                AdderBis.cx(A[2*m],B[2*m])  
                ###################################
                
                Adder.ccx(B[2*m],B[2*m+1],S[2*m+1])
                AdderBis.ccx(B[2*m],B[2*m+1],S[2*m+1])
                Buffer.append(S[2*m+1])
                P_inv_list.append([B[2*m],B[2*m+1],S[2*m+1]])       #Enregistre toutes les opérations à uncompute
            else:
                Adder.ccx(P_list[t-2][2*m-1],P_list[t-2][2*m],Anc[i])
                AdderBis.ccx(P_list[t-2][2*m-1],P_list[t-2][2*m],Anc[i])
                Buffer.append(Anc[i])
                Buffer_Anc_list.append([P_list[t-2][2*m-1],P_list[t-2][2*m],Anc[i]])
                i=i+1
        P_list.append(Buffer)
        if(t>1):
            Anc_list.append(Buffer_Anc_list)
            Buffer_Anc_list=[]
        Buffer=[]
        #Adder.barrier()
    
    
    #C-round/P1 uncompute/Recompute
    for t in range(log2_3n,0,-1):
        tmp=int(math.pow(2,t))
        tmp1=int(math.pow(2,t-1))
        roundUpperBound=math.floor((numberOfQubits-tmp1)/tmp)
        if(t==1):
            #si t==1 alors on uncompute tous les P1 et recompute P0,G0
            #pour pouvoir mettre les C1
            for i in range(1,len(P_inv_list)+1):
                Adder.ccx(P_inv_list[i-1][0],P_inv_list[i-1][1],P_inv_list[i-1][2])  #Uncompute les P1
                AdderBis.ccx(P_inv_list[i-1][0],P_inv_list[i-1][1],P_inv_list[i-1][2])  #Uncompute les P1
                Adder.cx(A[tmp*i+tmp1-1],B[tmp*i+tmp1-1])
                AdderBis.cx(A[tmp*i+tmp1-1],B[tmp*i+tmp1-1])
                Adder.ccx(A[tmp*i+tmp1-1],B[tmp*i+tmp1-1],S[tmp*i+tmp1])
                AdderBis.ccx(A[tmp*i+tmp1-1],B[tmp*i+tmp1-1],S[tmp*i+tmp1])
                Adder.cx(A[tmp*i+tmp1-1],B[tmp*i+tmp1-1]) 
                AdderBis.cx(A[tmp*i+tmp1-1],B[tmp*i+tmp1-1]) 
        else:
            for k in range(0,len(Anc_list[t-2])):
                Adder.ccx(Anc_list[t-2][k][0],Anc_list[t-2][k][1],Anc_list[t-2][k][2])
                AdderBis.ccx(Anc_list[t-2][k][0],Anc_list[t-2][k][1],Anc_list[t-2][k][2]) 
        for m in range (1,roundUpperBound+1):
            if t>1:
                Adder.ccx(S[tmp*m],P_list[t-2][2*m-1],S[tmp*m+tmp1])
                AdderBis.ccx(S[tmp*m],P_list[t-2][2*m-1],S[tmp*m+tmp1])
            else:
                Adder.ccx(S[tmp*m],B[2*m],S[tmp*m+tmp1])
                AdderBis.ccx(S[tmp*m],B[2*m],S[tmp*m+tmp1])
    
    #Adder.barrier()
    

    #Adder.barrier()
    #compute sum
    for i in range(0,numberOfQubits):
        Adder.cx(B[i],S[i])
        AdderBis.cx(B[i],S[i])
    Adder.cx(A[0],S[0])
    AdderBis.cx(A[0],S[0])
    
    #Adder.barrier()
    #uncompute b
    for i in range(1,numberOfQubits):
        Adder.cx(A[i],B[i])
        AdderBis.cx(A[i],B[i])
     
    #Adder.barrier()
    
    #adding a  Register to measure the state of the qubits
    outS = ClassicalRegister(numberOfQubits+1,"outS")

    #adding measure circuit
    meas = QuantumCircuit(S,outS)
    meas.barrier(S)
    meas.measure(S,outS)  #measur S values into outS
    qc = Adder + meas   #concatenate the circuit
    
    
    return qc

In [4]:
def fromBinaryStringToInt(binString):
    n=binString[0]
    bitString=[]
    bitInt=0
    a=0
    for i in range(0,len(n)):
        if(n[i]!=" "):
            bitString.append(n[i])
    for i in range(len(bitString)-1,-1,-1):
        if(bitString[i]=="1"):
            bitInt=bitInt+int(math.pow(2,a))
        a=a+1
    return(bitInt)

In [5]:
#Test bench
def testBench():
    nbTest=0
    backendQASM = BasicAer.get_backend('qasm_simulator')
    for i in range(1,8):
        bound = math.pow(2,i)
        stateVectorSize=int(bound)
        for j in range(0,int(stateVectorSize/2)):
            valueA = randint(1,stateVectorSize)
            valueB = randint(1,stateVectorSize)
            Adder = buildAdder(i,valueA,valueB)
            job = execute(Adder,backendQASM,shots=1)
            result = job.result()
            counts = result.get_counts(Adder)
            nbTest=nbTest+1
            print("nbTest : ")
            print(nbTest)
            print("n=")
            print(i)
            if(valueA+valueB-2!=fromBinaryStringToInt(list(counts.keys()))):
                print("ERROR WRONG SUM !")
                print(valueA)
                print(valueB)
                print(counts)
                return
    print("TEST COMPLETE")

In [130]:
testBench()

1
nbTest : 
1
n=
1
0
nbTest : 
2
n=
2
0
nbTest : 
3
n=
2
0
nbTest : 
4
n=
3
0
nbTest : 
5
n=
3
0
nbTest : 
6
n=
3
0
nbTest : 
7
n=
3
0
nbTest : 
8
n=
4
0
nbTest : 
9
n=
4
0
nbTest : 
10
n=
4
0
nbTest : 
11
n=
4
0
nbTest : 
12
n=
4
0
nbTest : 
13
n=
4
0
nbTest : 
14
n=
4
0
nbTest : 
15
n=
4
0
nbTest : 
16
n=
5
0
nbTest : 
17
n=
5
0
nbTest : 
18
n=
5
0
nbTest : 
19
n=
5
0
nbTest : 
20
n=
5
0
nbTest : 
21
n=
5
0
nbTest : 
22
n=
5
0
nbTest : 
23
n=
5
0
nbTest : 
24
n=
5
0
nbTest : 
25
n=
5
0
nbTest : 
26
n=
5
0
nbTest : 
27
n=
5
0
nbTest : 
28
n=
5
0
nbTest : 
29
n=
5
0
nbTest : 
30
n=
5
0
nbTest : 
31
n=
5
0
nbTest : 
32
n=
6
0
nbTest : 
33
n=
6
0
nbTest : 
34
n=
6
0
nbTest : 
35
n=
6
0
nbTest : 
36
n=
6
0
nbTest : 
37
n=
6
0
nbTest : 
38
n=
6
0
nbTest : 
39
n=
6
0
nbTest : 
40
n=
6
0
nbTest : 
41
n=
6
0
nbTest : 
42
n=
6
0


KeyboardInterrupt: 

In [133]:
va=100
vb=76
A=buildAdder(8,va+1,vb+1)

A.depth()

19

In [116]:
backendQASM = BasicAer.get_backend('qasm_simulator')
job = execute(A,backendQASM,shots=1)
result = job.result()
counts= result.get_counts(A)
print(va+vb)
print(counts)

101
{'111000': 1}
